### (week5)Sprint機械学習フロー
> `aim.`
> + 機械学習の実践的な流れを知る
> + 高い汎化性能を持つモデルを完成させる

>     Kaggleの Home Credit Default Risk コンペティションを題材に、機械学習の実践的な流れを学びます。
>     特に適切な 検証 を行い、高い 汎化性能 のあるモデルを完成させることを目指します。

> `Cf.`
> + [Home Credit Default Risk | Kaggle](https://www.kaggle.com/c/home-credit-default-risk)

##### - IMPORT

In [16]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Algorithm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Validation/Tuning
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV 

# Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

##### - Dataset

In [21]:
iris = load_iris()

iris_features = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_labels = pd.Series(iris.target)

display(iris_features.head(3))
display(iris_labels.head(3))

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


0    0
1    0
2    0
dtype: int64

In [18]:
X_train, X_test, y_train, y_train = train_test_split(
    iris_features, iris_labels, test_size=0.5, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_train.shape)

(75, 4) (75, 4) (75,) (75,)


##### - Baseline

In [19]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test) 

/Users/t/.pyenv/versions/3.8.5/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:06:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [22]:
y_pred_xgb

array([2, 1, 0, 0, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 0, 0, 2, 1, 0, 2, 0,
       1, 0, 2, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 0, 1, 1, 1, 2, 2,
       0, 2, 1, 0, 0, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 0, 1, 1,
       0, 0, 0, 0, 2, 0, 0, 0, 2])

#### 【問題1】クロスバリデーション
>      事前学習期間では検証データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）
>      しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。
>      分割を複数回行い、それぞれに対して学習と検証を行う方法です。
>      複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。
>      事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。


> `Cf.`
> + [sklearn.model_selection.KFold — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)
> + [比較的少なめのデータで機械学習する時は交差検証 (Cross Validation) をするのです - Qiita](https://qiita.com/LicaOka/items/c6725aa8961df9332cc7#%E4%BA%A4%E5%B7%AE%E6%A4%9C%E8%A8%BC-cross-validation-%E3%81%A8%E3%81%AF)
> + [Does TensorFlow have cross validation implemented for its users? - stackoverflow](https://stackoverflow.com/questions/38164798/does-tensorflow-have-cross-validation-implemented-for-its-users?rq=1)

##### - CrossValidation(KFold class)

In [36]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
    print("%s %s" % (train, test))

[2 3] [0 1]
[0 1] [2 3]


In [13]:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
score = cross_val_score(xgb, iris_features, iris_labels, cv=cv)
print(score.mean(), "\n", score)

/Users/t/.pyenv/versions/3.8.5/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:17:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:17:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:17:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:17:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

#### 【問題2】グリッドサーチ
>      これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。
>      パラメータの詳細は今後のSprintで学んでいくことになります。
>      機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。
>      最適なパラメータを探していくことを パラメータチューニング と呼びます。
>      パラメータチューニングをある程度自動化する単純な方法としては グリッドサーチ があります。
>      scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。
>      そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。
>      どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。

> `Cf.`
> + [sklearn.model_selection.GridSearchCV — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

>      GridSearchCVクラスには引数としてモデル、探索範囲、さらにクロスバリデーションを何分割で行うかを与えます。
>      クロスバリデーションの機能も含まれているため、これを使用する場合はKFoldクラスを利用する必要はありません。

#### 【問題3】Kaggle Notebooksからの調査
>      KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

> `Cf.`
> + [交差検証（cross validation／クロスバリデーション）の種類を整理してみた](https://aizine.ai/cross-validation0910/)

#### 【問題4】高い汎化性能のモデル作成
>     問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。
>     その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

#### 【問題5】最終的なモデルの選定
>     最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。
>     どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。